# Exploración de Datos

In [3]:
import sys
import pathlib

# Agrega la raíz del proyecto a sys.path
sys.path.append(str(pathlib.Path().resolve().parent))


In [16]:
import pandas as pd
df = pd.read_parquet("C:/Users/felip/market-ia-trading-bot/data/processed/AAPL.parquet", engine="fastparquet")
print(df[["net_buy","buy_count","sell_count","amount_net"]].tail(10))


            net_buy  buy_count  sell_count  amount_net
Date                                                  
2020-03-19      0.0        0.0         0.0         NaN
2020-03-20      1.0        1.0         0.0     15001.0
2020-03-23      0.0        0.0         0.0         NaN
2020-03-24      1.0        1.0         0.0     15001.0
2020-03-25      0.0        0.0         0.0         NaN
2020-03-26      0.0        0.0         0.0         NaN
2020-03-27      0.0        0.0         0.0         NaN
2020-03-30      0.0        0.0         0.0         NaN
2020-03-31      0.0        0.0         0.0         NaN
2020-04-01      0.0        0.0         0.0         NaN


In [9]:
import os
print("Working dir:", os.getcwd())


Working dir: c:\Users\felip\market-ia-trading-bot\notebooks


In [12]:
df.columns.tolist()


['SimFinId',
 'Currency_x',
 'Fiscal Year',
 'Fiscal Period_x',
 'Publish Date_x',
 'Restated Date_x',
 'Shares (Basic)_x',
 'Shares (Diluted)_x',
 'Revenue',
 'Cost of Revenue',
 'Gross Profit',
 'Operating Expenses',
 'Selling, General & Administrative',
 'Research & Development',
 'Depreciation & Amortization_x',
 'Operating Income (Loss)',
 'Non-Operating Income (Loss)',
 'Interest Expense, Net',
 'Pretax Income (Loss), Adj.',
 'Abnormal Gains (Losses)',
 'Pretax Income (Loss)',
 'Income Tax (Expense) Benefit, Net',
 'Income (Loss) from Continuing Operations',
 'Net Extraordinary Gains (Losses)',
 'Net Income',
 'Net Income (Common)',
 'Currency_y',
 'Fiscal Period_y',
 'Publish Date_y',
 'Restated Date_y',
 'Shares (Basic)_y',
 'Shares (Diluted)_y',
 'Cash, Cash Equivalents & Short Term Investments',
 'Accounts & Notes Receivable',
 'Inventories',
 'Total Current Assets',
 'Property, Plant & Equipment, Net',
 'Long Term Investments & Receivables',
 'Other Long Term Assets',
 'Tota

In [1]:
import pandas as pd
df = pd.read_parquet("C:/Users/felip/market-ia-trading-bot/data/processed/AAPL.parquet", engine="fastparquet")
print(df.columns[:20])                    # mira qué columnas hay
print(df[["daily_return","net_buy","sentiment_score"]].tail(5))
print(df.index.min(), df.index.max(), len(df))


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'volume_adi',
       'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_vwap', 'volume_mfi', 'volume_nvi',
       'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw'],
      dtype='object')
            daily_return  net_buy  sentiment_score
Date                                              
2020-03-26      0.052623      0.0              0.0
2020-03-27     -0.041402      0.0              0.0
2020-03-30      0.028538      0.0              0.0
2020-03-31     -0.002041      0.0              0.0
2020-04-01     -0.052617      0.0              0.0
1980-12-12 00:00:00 2020-04-01 00:00:00 9909
